# Ejemplo archivos
1. Creamos a SparkSession.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession.builder\
  .appName("arquivo-example-1")\
  .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
  .getOrCreate()

print("Versión de spark: ",spark.version)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ed7044ec-fc8c-4b26-b88f-047623f62f6c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

Versión de spark:  3.5.7


2. Obtéñese o esquema e indícase a orixe dos datos para o procesamento en *streaming*, neste caso un cartafol con ficheiros *json*.


In [2]:
# Defínese a ruta onde se atopan os ficheiros JSON cos datos de entrada
path = "/data/flight-data/json"

# Lense os datos de forma estática para obter o esquema
static = spark.read.json(path)

# Extráese o esquema do DataFrame estático
dataSchema = static.schema

# Créase un DataFrame de streaming usando o esquema obtido previamente
# maxFilesPerTrigger = 1 indica que se procesará como máximo un ficheiro novo en cada micro-batch
streaming = spark.readStream \
    .schema(dataSchema) \
    .option("maxFilesPerTrigger", 1) \
    .json(path)

# Móstrase o esquema do DataFrame de streaming
streaming.printSchema()


root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



3. Obtense o DataFrame de saída transformando os datos iniciais. Neste caso agrúpase por *DEST_COUNTRY_NAME* e cóntase.


In [3]:
counts = streaming.groupBy("DEST_COUNTRY_NAME").count()

4. Iniciamos o procesamento en streaming con saída a memoria.

In [4]:
query = (
    counts.writeStream
        # Asígnase un nome á consulta
        .queryName("counts")
        # Indícase que a saída se gardará en memoria
        .format("memory")
        # Escríbese o resultado completo en cada micro-batch
        .outputMode("complete")
        # Iníciase a consulta de streaming
        .start()
)


[Stage 2:================================================>        (54 + 1) / 64]

5. Mostramos los datos:

In [5]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM counts").show()
    sleep(5)

[Stage 2:===================================================>     (58 + 1) / 64]

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
+-----------------+-----+



+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|           Nicaragua|    1|
|         South Korea|    1|
|   Equatorial Guinea|    1|
|          Guadeloupe|    1|
|           Australia|    1|
|       French Guiana|    1|
|              Turkey|    1|
|         The Bahamas|    1|
|Netherlands Antilles|    1|
|             Bermuda|    1|
| Antigua and Barbuda|    1|
|             Denmark|    1|
|               Samoa|    1|
|         Switzerland|    1|
| Trinidad and Tobago|    1|
|             Vietnam|    1|
|         Philippines|    1|
|               Palau|    1|
|             Belgium|    1|
|           Hong Kong|    1|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|   Equatorial Guinea|    1|
|           Nicaragua|    2|
|         South Korea|    2|
|                Togo|    1|
|          Guadeloupe|    2|
|           Australia|    2|
|       French Guiana|    2|
|             Romania|    1|
|              Turkey|    2|
|         The Bahamas|    2|
|Netherlands Antilles|    1|
|             Bermuda|    2|
| Antigua and Barbuda|    2|
|             Denmark|    2|
|               Samoa|    2|
|         Switzerland|    2|
|             Vietnam|    2|
| Trinidad and Tobago|    2|
|         Philippines|    2|
|               Palau|    2|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|   Equatorial Guinea|    1|
|           Nicaragua|    3|
|         South Korea|    3|
|                Togo|    1|
|            Cameroon|    1|
|          Guadeloupe|    3|
|           Australia|    3|
|       French Guiana|    3|
|             Romania|    1|
|              Turkey|    3|
|         The Bahamas|    3|
|Netherlands Antilles|    1|
|             Bermuda|    3|
| Antigua and Barbuda|    3|
|             Denmark|    3|
|               Samoa|    3|
|         Switzerland|    3|
|             Vietnam|    3|
| Trinidad and Tobago|    3|
|         Philippines|    3|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|   Equatorial Guinea|    2|
|           Nicaragua|    4|
|         South Korea|    4|
|                Togo|    1|
|            Cameroon|    1|
|          Guadeloupe|    4|
|           Australia|    4|
|       French Guiana|    4|
|           Indonesia|    1|
|             Romania|    2|
|              Turkey|    4|
|         The Bahamas|    4|
|Netherlands Antilles|    1|
|             Bermuda|    4|
| Antigua and Barbuda|    4|
|             Denmark|    4|
|              Rwanda|    1|
|               Samoa|    4|
|         Switzerland|    4|
|             Vietnam|    3|
+--------------------+-----+
only showing top 20 rows

